In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
import numpy as np
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint
from collections import Counter

from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras import losses

In [13]:
df = pd.read_csv("../data/cleaned_text_data.csv")
df.head()

,sentiment,message
0,Negative,upset update facebook texting might cry result...
1,Negative,dived many time ball managed save 50 rest go b...
2,Negative,whole body feel itchy like fire
3,Negative,behaving mad see
4,Negative,whole crew


In [14]:
train_data,test_data = train_test_split(df,test_size=0.2,random_state=11)


In [15]:
len(train_data)

1279999

In [16]:
# max_length = train_data['message'].apply(len).max()
# # print("Maximum message length:", max_length)
# # 

In [17]:
len(test_data)


320000

In [18]:
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)
train_data['message'] = train_data['message'].astype(str)
train_data['sentiment'] = train_data['sentiment'].astype(str)
test_data['message'] = test_data['message'].astype(str)
test_data['sentiment'] = test_data['sentiment'].astype(str)


In [65]:
max_words = 20000
oov_token = "unknown"

tokenizer = Tokenizer(num_words=max_words, oov_token=oov_token)
tokenizer.fit_on_texts(train_data['message'])

tokenizer_json = tokenizer.to_json()
with open('tokenizer.json', 'w') as json_file:
    json_file.write(tokenizer_json)

In [20]:
print(str(tokenizer.texts_to_sequences(['There seems to be something wrong with you today'])))

[[5590, 390, 5590, 5590, 123, 304, 5590, 5590, 9]]


In [21]:
X_train, X_valid, y_train, y_valid= train_test_split(train_data['message'].tolist(),
                                                      train_data['sentiment'].tolist(),
                                                      test_size=0.1,
                                                      stratify = train_data['sentiment'].tolist(),
                                                      random_state=8)

#further divided traindata into train data and validation data (validation data will just be used to make sure that the model is training correctly.)

In [22]:
print('Train data len:'+str(len(X_train)))
print('Class distribution'+str(Counter(y_train)))
print('Valid data len:'+str(len(X_valid)))
print('Class distribution'+ str(Counter(y_valid)))

Train data len:1151999
Class distributionCounter({'Positive': 576360, 'Negative': 575639})
Valid data len:128000
Class distributionCounter({'Positive': 64040, 'Negative': 63960})


In [23]:

train_sequences = tokenizer.texts_to_sequences(X_train)
valid_sequences = tokenizer.texts_to_sequences(X_valid)
test_sequences = tokenizer.texts_to_sequences(test_data['message'].tolist())

X_train = pad_sequences(train_sequences, padding='post', maxlen=50)
x_valid = pad_sequences(valid_sequences, padding='post', maxlen=50)
x_test = pad_sequences(test_sequences, padding='post', maxlen=50)



X_train = np.array(X_train)
x_valid = np.array(x_valid)
x_test = np.array(x_test)
print(X_train.shape)
print(x_valid.shape)
print(x_test.shape)


(1151999, 50)
(128000, 50)
(320000, 50)


In [24]:
le = LabelEncoder()

train_labels = le.fit_transform(y_train)
train_labels = np.asarray( tf.keras.utils.to_categorical(train_labels))

valid_labels = le.transform(y_valid)
valid_labels = np.asarray( tf.keras.utils.to_categorical(valid_labels))

test_labels = le.transform(test_data['sentiment'].tolist())
test_labels = np.asarray(tf.keras.utils.to_categorical(test_labels))



train_ds = tf.data.Dataset.from_tensor_slices((X_train,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid,valid_labels))
test_ds = tf.data.Dataset.from_tensor_slices((x_test,test_labels))

2024-06-09 10:39:44.394397: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 230399800 exceeds 10% of free system memory.


In [25]:
X_train[0]

array([4201, 1356, 5590,  105,   40,    4,  223, 1856,  154,   58,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [37]:
# Assuming train_data['sentiment'] contains all possible classes
le = LabelEncoder()
le.fit(train_data['sentiment'].tolist())

# Transform labels
train_labels = le.transform(y_train)
train_labels = np.asarray(tf.keras.utils.to_categorical(train_labels))

valid_labels = le.transform(y_valid)
valid_labels = np.asarray(tf.keras.utils.to_categorical(valid_labels))

test_labels = le.transform(test_data['sentiment'].tolist())
test_labels = np.asarray(tf.keras.utils.to_categorical(test_labels))

# Verify the number of classes
num_classes = len(le.classes_)
print("Number of classes:", num_classes)


Number of classes: 2


In [38]:


number_of_classes = 2  

max_features = 20000
embedding_dim = 64
sequence_length = 50

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features + 1, embedding_dim, input_length=sequence_length,
                                    embeddings_regularizer=regularizers.l2(0.0005)))

model.add(tf.keras.layers.Conv1D(128, 3, activation='relu',
                                 kernel_regularizer=regularizers.l2(0.0005),
                                 bias_regularizer=regularizers.l2(0.0005)))

model.add(tf.keras.layers.GlobalMaxPooling1D())

model.add(tf.keras.layers.Dropout(0.5))


model.add(tf.keras.layers.Dense(number_of_classes, activation='sigmoid',
                                kernel_regularizer=regularizers.l2(0.001),
                                bias_regularizer=regularizers.l2(0.001)))

model.summary()


model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), 
              optimizer='Nadam', 
              metrics=["CategoricalAccuracy"])


/home/aryn/.local/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_6          │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [40]:
epochs = 20
history = model.fit(train_ds.shuffle(2000).batch(128),
                    epochs= epochs ,
                    validation_data=valid_ds.batch(128),
                    verbose=1)

Epoch 1/20


9000/9000 ━━━━━━━━━━━━━━━━━━━━ 765s 85ms/step - CategoricalAccuracy: 0.7544 - loss: 0.5606 - val_CategoricalAccuracy: 0.7652 - val_loss: 0.5397
Epoch 2/20
9000/9000 ━━━━━━━━━━━━━━━━━━━━ 786s 87ms/step - CategoricalAccuracy: 0.7636 - loss: 0.5439 - val_CategoricalAccuracy: 0.7695 - val_loss: 0.5350
Epoch 3/20
9000/9000 ━━━━━━━━━━━━━━━━━━━━ 791s 88ms/step - CategoricalAccuracy: 0.7665 - loss: 0.5399 - val_CategoricalAccuracy: 0.7705 - val_loss: 0.5319
Epoch 4/20
9000/9000 ━━━━━━━━━━━━━━━━━━━━ 800s 89ms/step - CategoricalAccuracy: 0.7665 - loss: 0.5387 - val_CategoricalAccuracy: 0.7691 - val_loss: 0.5344
Epoch 5/20
9000/9000 ━━━━━━━━━━━━━━━━━━━━ 792s 88ms/step - CategoricalAccuracy: 0.7670 - loss: 0.5384 - val_CategoricalAccuracy: 0.7703 - val_loss: 0.5322
Epoch 6/20
9000/9000 ━━━━━━━━━━━━━━━━━━━━ 775s 86ms/step - CategoricalAccuracy: 0.7667 - loss: 0.5383 - val_CategoricalAccuracy: 0.7707 - val_loss: 0.5316
Epoch 7/20
9000/9000 ━━━━━━━━━━━━━━━━━━━━ 781s 87ms/step - CategoricalAccuracy: 0

In [44]:
model.save('sentimentmodel.keras')  


In [45]:
model.save('sentimentmodel.h5') 


In [63]:

import json 
tokenizer = Tokenizer(num_words=max_features)  
json_string = tokenizer.to_json()
with open('tokenizer.json', 'w') as outfile:
    outfile.write(json_string)

In [62]:
tokenizer

Generate predictions for all samples
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 36s 4ms/step
[[0.4569431  0.54305685]
 [0.4569431  0.54305685]
 [0.4569431  0.54305685]
 ...
 [0.4569431  0.54305685]
 [0.4569431  0.54305685]
 [0.4569431  0.54305685]]
